In [ ]:
# export
from fastai2.basics import *

In [ ]:
# default_exp pytorch.transformer

# Pytorch Transformer
> Utils about Transformer of pytorch

## Helpers

### gen_key_padding_mask

In [ ]:
# export
def gen_key_padding_mask(input_ids, pad_id):
    ''' Returns ByteTensor where True values are positions that contain pad_id.
        input_ids: (bs, seq_len) returns: (bs, seq_len)
    '''
    device = input_ids.device
    mask = torch.where(input_ids == pad_id, torch.tensor(1, device=device), torch.tensor(0, device=device)).to(device)
    return mask.bool()

In [ ]:
input_ids = torch.tensor([[12, 11, 0, 0], 
                          [9, 1, 5, 0]])
key_padding_mask = gen_key_padding_mask(input_ids, 0)
test_eq(key_padding_mask, torch.tensor([[0, 0, 1, 1],
                                        [0, 0, 0, 1]]).bool())

### gen_lm_mask

In [ ]:
# export
def gen_lm_mask(tgt_seq_len, device):
    """Generate a square mask for the sequence. The masked positions are filled with float('-inf').
        Unmasked positions are filled with float(0.0).
        ex: tgt_seq_len = 4
        [[0., -inf, -inf, -inf],
        [0., 0., -inf, -inf],
        [0., 0., 0., -inf],
        [0., 0., 0., 0.]])
    """
    mask = (torch.triu(torch.ones(tgt_seq_len, tgt_seq_len)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask.to(device)

In [ ]:
lm_mask = gen_lm_mask(4, 'cpu')
test_eq(lm_mask, torch.tensor([ [0., float('-inf'), float('-inf'), float('-inf')],
                                [0., 0., float('-inf'), float('-inf')],
                                [0., 0., 0., float('-inf')],
                                [0., 0., 0., 0.]]) )

## Modules

### BatchFirstTransformerEncoder

In [ ]:
# export
class BatchFirstTransformerEncoder(nn.TransformerEncoder):
    '''
        nn.TransformerEncoder want src be (seq_len, bs, embeded_size) and returns (seq_len, bs, embeded_size),
        just change it to accept batch first input and returns
    '''
    def forward(self, src, *inputs, **kwargs):
        ''' src: (bs, enc_seq_len, embeded_size), returns: (bs, enc_seq_len, embeded_size) '''
        src = src.permute(1, 0, 2) # (enc_seq_len, bs, embeded_size)
        output = super().forward(src, *inputs, **kwargs) # (enc_seq_len, bs, embeded_size)
        return output.permute(1, 0, 2) # (bs, enc_seq_len, embeded_size)

In [ ]:
encoder_layer = nn.TransformerEncoderLayer(d_model=128, nhead=2)
encoder_norm = nn.LayerNorm(normalized_shape=128)
encoder = BatchFirstTransformerEncoder(encoder_layer=encoder_layer, num_layers=2, norm=encoder_norm)

src = torch.randn((16, 50, 128)) # (bs, enc_seq_len, embeded_size)
output = encoder(src) # (bs, enc_seq_len, embeded_size)
test_eq(output.shape, (16, 50, 128))

### BatchFirstTransformerDecoder

In [ ]:
# export
class BatchFirstTransformerDecoder(nn.TransformerDecoder):
    '''
        nn.TransformerDecoder want tgt be (seq_len, bs, embeded_size) and returns (seq_len, bs, embeded_size),
        just change it to accept batch first input and returns
    '''
    def forward(self, tgt, memory, *inputs, **kwargs):
        ''' 
            tgt: (bs, dec_seq_len, embeded_size)
            memory: (bs, enc_seq_len, embeded_size)
            returns: (bs, dec_seq_len, embeded_size) 
        '''
        tgt = tgt.permute(1, 0, 2) # (dec_seq_len, bs, embeded_size)
        memory = memory.permute(1, 0, 2) # (enc_seq_len, bs, embeded_size)
        output = super().forward(tgt, memory, *inputs, **kwargs) # (dec_seq_len, bs, embeded_size)
        return output.permute(1, 0, 2) # (bs, dec_seq_len, embeded_size)

In [ ]:
decoder_layer = nn.TransformerDecoderLayer(d_model=128, nhead=2)
decoder_norm = nn.LayerNorm(normalized_shape=128)
decoder = BatchFirstTransformerDecoder(decoder_layer, num_layers=2, norm=decoder_norm)

tgt = torch.randn((16, 40, 128)) # (bs, dec_seq_len)
memory = torch.randn((16, 50, 128)) # (bs, enc_seq_len, embeded_size)
output = decoder(tgt, memory) # (bs, dec_seq_len, embeded_size)
test_eq(output.shape, (16, 40, 128))

### BatchFirstMultiheadAttention

In [ ]:
# export
class BatchFirstMultiheadAttention(nn.MultiheadAttention):
    '''
        Pytorch wants your query, key, value be (seq_len, b, embed_dim) and return (seq_len, b, embed_dim)
        But I like batch-first thing. input: (b, seq_len, embed_dim) output: (b, seq_len, embed_dim)
    '''
    def forward(self, query, key, value, **kwargs):
        '''
        - inputs:
        - query: (bs, tgt_seq_len, embed_dim)
        - key: (bs, src_seq_len, embed_dim)
        - value: (bs, src_seq_len, embed_dim)

        - outputs:
        - attn_output: (bs, tgt_seq_len, embed_dim)
        - attn_weight: (bs, tgt_seq_len, src_seq_len), Averaged weights that averaged over all heads
        '''
        attn_output, attn_weight = super().forward(query.permute(1, 0, 2), key.permute(1, 0, 2), value.permute(1, 0, 2), **kwargs)
        return attn_output.permute(1, 0, 2), attn_weight

In [ ]:
multi_attn = BatchFirstMultiheadAttention(embed_dim=128, num_heads=1, dropout=0)
query = torch.randn((3, 40, 128))
key = torch.randn((3, 50, 128))
value = torch.randn((3, 50, 128))
attn_output, attn_weight = multi_attn(query, key, value)
test_eq(attn_output.shape, (3, 40, 128))
test_eq(attn_weight.shape, (3, 40, 50))

## CrossAttention

In [ ]:
# export
class CrossAttention(nn.Module):
    def __init__(self, embed_dim, num_heads=1, drop_p=0, num_layers=1):
        super().__init__()
        self.cross_attn_layers = nn.ModuleList(
            [BatchFirstMultiheadAttention(embed_dim, num_heads=num_heads, dropout=drop_p) for _ in range(num_layers)]
        )
    def forward(self, tgt, src, src_key_padding_mask):
        '''
        tgt: (bs, tgt_seq_len, embed_size)
        src: (bs, src_seq_len, embed_size)
        src_key_padding_mask: (bs, src_seq_len)
        returns: output, attn_weight
            output: (bs, tgt_seq_len, embed_dim)
            attn_weight: (bs, tgt_seq_len, src_seq_len)
        '''
        for layer in self.cross_attn_layers:
            tgt, attn_weight = layer(tgt, src, src, key_padding_mask=src_key_padding_mask)
        return tgt, attn_weight

In [ ]:
tgt = torch.randn((3, 40, 768))
src = torch.randn((3, 50, 768))
src_key_padding_mask = torch.zeros((3, 50)).bool()
cross_attn = CrossAttention(embed_dim=768, num_layers=2)

cross_attn_out, cross_attn_weight = cross_attn(tgt, src, src_key_padding_mask)
test_eq(cross_attn_out.shape, (3, 40, 768))
test_eq(cross_attn_weight.shape, (3, 40, 50))

## Export -

In [ ]:
# hide
from nbdev.export import notebook2script
notebook2script()

Converted 01_data.core.ipynb.
Converted 02_pytorch.transformer.ipynb.
Converted 03_pytorch.model.ipynb.
Converted 04_optuna.ipynb.
Converted index.ipynb.
